<a href="https://colab.research.google.com/github/richardan01/fairprice_stock_analysis/blob/main/Look_ecommerce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Authenticate this session and choose project
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
import ipywidgets as widgets
%reload_ext google.colab.data_table
auth.authenticate_user()
print("Authenticated.")

project_list = !gcloud projects list
project_ids = [row.split()[0] for row in project_list[1:]]
dummy_option = "(Change here if needed)"
options = project_ids.copy()
options.insert(0, dummy_option)

if 'fairprice-bigquery' in project_ids:
  project_id = "fairprice-bigquery"
  client = bigquery.Client(project=project_id)
else:
  project_id = project_ids[0]
  client = bigquery.Client(project=project_id)

print(f"Default project: `{project_id}` as of " + str(pd.Timestamp.now('Singapore')))

dropdown = widgets.Dropdown(
    options=options,
    value=dummy_option,
    description='Change? ',
)

def on_change(change):
  global project_id
  if change['name'] == 'value' and change['new'] in project_ids:
    project_id = change['new']
    client = bigquery.Client(project=project_id)
    print(f"Current project: `{project_id}` as of " + str(pd.Timestamp.now('Singapore')))

dropdown.observe(on_change)
display(dropdown)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-1-584614823.py", line 3, in <cell line: 0>
    from google.cloud import bigquery
  File "/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/__init__.py", line 36, in <module>
    from google.cloud.bigquery.client import Client
  File "/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/client.py", line 72, in <module>
    from google.cloud.bigquery import _job_helpers
  File "/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/_job_helpers.py", line 48, in <module>
    from google.cloud.bigquery import job
  File "/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/job/__init__.py", line 27, in <module>
    from google.cloud.bigquery.job.copy_ import CopyJob
  File "/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/

TypeError: object of type 'NoneType' has no len()

# 1.0 Sales Analysis

## **1.1 Sales Revenue by Category**

The table below shows total sales revenue of each product category available in Look Ecommerce

In [ ]:

%%bigquery --project $project_id
SELECT
  products.category,
  SUM(order_items.sale_price) AS total_sales,
  COUNT(DISTINCT order_items.user_id) AS total_customers
FROM
  `bigquery-public-data`.`thelook_ecommerce`.`order_items` AS order_items
INNER JOIN
  `bigquery-public-data`.`thelook_ecommerce`.`products` AS products
ON
  order_items.product_id = products.id
GROUP BY
  1
ORDER BY
  total_sales DESC;

- The most profitable product category is Outwear & Coats, followed by Jeans and Sweaters.
- The least profitable product categories are clothing sets, Jumpsuits & Rompers.

## **1.2 Sales Revenue percentage**

The table below shows revenue percentage in the ECommerce Store

In [ ]:
%%bigquery --project $project_id
-- total revenue by category and percentage
SELECT
  category,
  SUM(sale_price) AS total_revenue,
  ROUND((SUM(sale_price) / (
      SELECT
        SUM(sale_price)
      FROM
        `bigquery-public-data`.`thelook_ecommerce`.`order_items`)) * 100, 2) AS revenue_percentage
FROM
  `bigquery-public-data`.`thelook_ecommerce`.`order_items` AS order_items
LEFT JOIN
  `bigquery-public-data`.`thelook_ecommerce`.`products` AS products
ON
  order_items.product_id = products.id
GROUP BY
  category;

The top revenue driver is the **"Outwear and Coats"** category, which accounts for 12% of total sales. This indicates a strong product-market fit within the outerwear and coats segment.


## 1.3 Brand by revenue

The table below shows the revenue of brands sell in Look Ecommerce

In [ ]:
%%bigquery --project $project_id
SELECT
  p.brand AS brand,
  ROUND(SUM(oi.sale_price * o.num_of_item), 2) AS revenue,
  SUM(o.num_of_item) AS quantity
FROM
  `bigquery-public-data`.`thelook_ecommerce`.`order_items` AS oi
INNER JOIN
  `bigquery-public-data`.`thelook_ecommerce`.`orders` AS o
ON
  oi.order_id = o.order_id
INNER JOIN
  `bigquery-public-data`.`thelook_ecommerce`.`products` AS p
ON
  oi.product_id = p.id
WHERE
  oi.status NOT IN ('Cancelled',
    'Returned')
GROUP BY
  1
ORDER BY
  revenue DESC;

**Analysis**
1. Calvin Klein brought in the most revenue.
2. Allegra K had the most sold quantity, double the amount sold by Calvin Klein.
3. Marshal Brand brought in the least revenue.

## 1.4 Sales Growth Over year
The table below shows the times series for sales growth over time


In [ ]:
# @title
%%bigquery --project $project_id
SELECT
  order_year,
  total_sales,
  ROUND((total_sales - LAG(total_sales, 1, 0) OVER (ORDER BY order_year)) * 100.0 / LAG(total_sales, 1, 1) OVER (ORDER BY order_year), 2) AS yoy_growth_percentage
FROM (
  SELECT
    EXTRACT(YEAR
    FROM
      DATE(delivered_at)) AS order_year,
    SUM(sale_price) AS total_sales
  FROM
    `bigquery-public-data`.`thelook_ecommerce`.`order_items`
  WHERE
    delivered_at IS NOT NULL
    AND returned_at IS NULL
    AND status = 'Complete'
  GROUP BY
    1 );

**Analysis**

1. Big Y0Y% growth between 2019 to 2020
2. There is a decline bettween 2021 to 2022 but revenue start to increase back from 2023 to 2024



# **2.0 Profitable Analysis**


## 2.1 Overall Profitability over time

In [ ]:
%%bigquery --project $project_id
SELECT
  EXTRACT(YEAR
  FROM
    created_at) AS year,
  round(SUM(sale_price)) AS total_revenue,
  round(SUM(cost)) AS total_cost,
  round(SUM(sale_price - cost)) AS gross_profit
FROM
  `bigquery-public-data`.`thelook_ecommerce`.`order_items` AS order_items
LEFT JOIN
  `bigquery-public-data`.`thelook_ecommerce`.`products` AS products
ON
  order_items.product_id = products.id
GROUP BY
  1;

Operating profit trippled in 2020 due to increased revenue and cost efficiency.

## 2.2 **Profit margin by product category**


In [ ]:
%%bigquery --project $project_id
SELECT
  products.category,
  SUM(order_items.sale_price) AS total_revenue,
  SUM(order_items.sale_price - products.cost) AS total_profit,
  SUM(order_items.sale_price - products.cost) / SUM(order_items.sale_price) AS profit_margin,
  SUM(order_items.id) AS quantity_sold
FROM
  `bigquery-public-data`.`thelook_ecommerce`.`products` AS products
INNER JOIN
  `bigquery-public-data`.`thelook_ecommerce`.`order_items` AS order_items
ON
  products.id = order_items.product_id
GROUP BY
  1;



This table shows the product profit margin

## 2.3 Recommendation for Profitability
**1. Increase Sales in High-Margin Products**


- From the profitability_improvement_analysis, we can identify which products have the highest gross margin percentages.
- Focusing marketing efforts and promotions on these products can drive higher overall profitability.


**2. Boost E-commerce Growth**

- Online channels are underperforming compared to in-store sales but typically offer higher profit margins due to lower operational costs (e.g., rent, staff).


**3. Reduce Operational Costs**

- Streamlining supply chain operations, improving inventory management, and negotiating better terms with suppliers can enhance gross margins.

# 3.0 Customer Behavior Analysis

## 3.1 Customer Segmentation based on age **group**

In [ ]:
%%bigquery --project $project_id
-- group it based on kids, teenagers, adults, elderly
SELECT
  CASE
    WHEN users.age BETWEEN 0 AND 12 THEN 'Kids'
    WHEN users.age BETWEEN 13
  AND 19 THEN 'Teenagers'
    WHEN users.age BETWEEN 20 AND 64 THEN 'Adults'
    ELSE 'Elderly'
END
  AS age_group,
  users.gender,
  COUNT(DISTINCT users.id) AS customer_count,
  SUM(orders.num_of_item) AS total_items_ordered,
  SUM(order_items.sale_price) AS total_revenue
FROM
  `bigquery-public-data`.`thelook_ecommerce`.`users` AS users
INNER JOIN
  `bigquery-public-data`.`thelook_ecommerce`.`orders` AS orders
ON
  users.id = orders.user_id
INNER JOIN
  `bigquery-public-data`.`thelook_ecommerce`.`order_items` AS order_items
ON
  orders.order_id = order_items.order_id
GROUP BY
  1,
  2;

**Insight**
1. Most revenue is done by mostly Adults and Teenagers and kids the least
2. relatively low numbers in elderly and kids group

## 3.2 Customer Segmentation by Gender

In [ ]:
%%bigquery --project $project_id
-- customer segmentation by gender total revenue
SELECT
  users.gender,
  SUM(order_items.sale_price) AS total_revenue
FROM
  `bigquery-public-data`.`thelook_ecommerce`.`users` AS users
INNER JOIN
  `bigquery-public-data`.`thelook_ecommerce`.`orders` AS orders
ON
  users.id = orders.user_id
INNER JOIN
  `bigquery-public-data`.`thelook_ecommerce`.`order_items` AS order_items
ON
  orders.order_id = order_items.order_id
GROUP BY
  1;


Male customers generated more revenue and bought more products in quantity than female customers.

## 3.3 Customer Behavior Analysis : Total Customers by Country

In [ ]:
%%bigquery --project $project_id
SELECT
  users.country,
  SUM(order_items.sale_price) AS total_revenue
FROM
  `bigquery-public-data`.`thelook_ecommerce`.`users` AS users
INNER JOIN
  `bigquery-public-data`.`thelook_ecommerce`.`order_items` AS order_items
ON
  users.id = order_items.user_id
GROUP BY
  1
ORDER BY
  total_revenue DESC;

## 3.4 Customer Behavior Analysis : Marketing Traffic Source

In [ ]:
%%bigquery --project $project_id
SELECT
  traffic_source,
  COUNT(*) AS total_customers
FROM
  `bigquery-public-data`.`thelook_ecommerce`.`users`
GROUP BY
  traffic_source;

## 3.4 Recommendation to grow customer base
**1. Enhance Digital Marketing**
Focus on underperforming online channels by creating targeted ad campaigns, improving SEO, and enhancing the user experience.

**2. Implement Loyalty Programs:**
Reward high-value customers to encourage repeat purchases and increase customer lifetime value (CLV).

**3. Promotions During Off-Peak Periods: **
Implement marketing campaigns to drive traffic and sales during typically slower months.

# 4.0 Inventory Analysis

## 4.1 Inventory and overstock analysis

In [ ]:
%%bigquery --project $project_id
-- group it by prodcut category
SELECT
  t0.product_category,
  COUNT(t0.id) AS total_inventory,
  COUNT(CASE
      WHEN t0.sold_at IS NULL THEN t0.id
  END
    ) AS unsold_inventory,
  COUNT(CASE
      WHEN t0.sold_at IS NOT NULL THEN t0.id
  END
    ) AS sold_inventory
FROM
  `bigquery-public-data`.`thelook_ecommerce`.`inventory_items` AS t0
GROUP BY
  1;

**Analysis**

1. **Intimates** leads with around 36k total units. This indicates a major investment in this category but also a large proportion of unsold items.

2. **Jeans, Tops & Tees, Hoodies & Sweatshirts** also have high total inventory counts (~32k–34k).

3. **Outerwear & Coats and Accessories** appear to have relatively lower total inventory (in the mid-20k range), which could be strategic (due to higher cost per unit) or a result of stronger turnover.

# 5.0 Recommendation to improve the business

Based on the the  analysis across Sales, Profitability, Customer Behavior, and Inventory Management, here is some of the few recommendation to enchance business profitability:

**1. Enhance E-commerce Performance**

- Invest in SEO, digital marketing, user experience improvements, and a stronger mobile presence will attract more customers to the online channel.

- Implement online promotions and improving checkout processes can increase conversion rates and overall profitability.

**2. Optimize Inventory Management**

- From the inventory analysis, several products are overstocked while others frequently run out of stock.
- YOu can product bundle some product through promotion campaign or seasonal campaign to enchance profitabilty

**3. Implement Customer Loyalty Programs **

- Reward high-value customers identified from the segmentation analysis with  promotions, rewards, or  loyalty programs can increase customer retention and repeat purchases.

